In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, matthews_corrcoef
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
#import tensorflow_model_optimization as tfmot
from tensorflow.keras.layers import BatchNormalization

# Check if GPU is available
if not tf.test.gpu_device_name():
    print('GPU device not found. Using CPU.')
else:
    print('GPU found. Using GPU.')


# Defining the deep learning model
def create_transfer_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers[:-4]:
        layer.trainable = True

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(BatchNormalization())  # Add Batch Normalization
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    return model

# Defining input shape and number of classes
input_shape = (150, 150, 3)
num_classes = 1

# Creating the model
model = create_transfer_model(input_shape, num_classes)

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Setting up data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'content/DataSet/train',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    'content/DataSet/test',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Learning Rate Schedule
def lr_schedule(epoch):
    initial_lr = 0.0001
    if epoch < 5:
        return initial_lr
    else:
        return initial_lr * np.exp(0.1 * (5 - epoch))

lr_scheduler = LearningRateScheduler(lr_schedule)

# Callbacks for early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/save/best_model.keras', save_best_only=True)

# Training the model with callbacks
history = model.fit(
    train_generator,
    epochs=150,
    validation_data=validation_generator,
    callbacks=[lr_scheduler, early_stopping, model_checkpoint]
)

# Save the final model
model.save('/content/drive/MyDrive/save/final_dental_caries_detection.keras')

# Evaluate the model on the validation set
validation_generator.reset()
y_true = validation_generator.classes
y_pred_prob = model.predict(validation_generator)
y_pred = np.round(y_pred_prob)  # Round probabilities to get binary predictions

# Confusion Matrix
conf_mat = confusion_matrix(y_true, y_pred)

# Calculate Sensitivity and Specificity
tn, fp, fn, tp = conf_mat.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f'Sensitivity: {sensitivity:.4f}')
print(f'Specificity: {specificity:.4f}')

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_true, y_pred)
print(f'Matthews Correlation Coefficient: {mcc:.4f}')

# Plotting the Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=['non-caries', 'caries'], yticklabels=['non-caries', 'caries'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Normalized Confusion Matrix
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=['non-caries', 'caries'], yticklabels=['non-caries', 'caries'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Normalized Confusion Matrix')
plt.show()

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve with AUC value
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# Print Classification Report
class_labels = ['non-caries', 'caries']
print(classification_report(y_true, y_pred, target_names=class_labels))

# Post-Training Quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized model
with open('/content/drive/MyDrive/save/quantized_dental_caries_detection.tflite', 'wb') as f:
    f.write(quantized_tflite_model)


GPU device not found. Using CPU.
Found 1260 images belonging to 2 classes.
Found 294 images belonging to 2 classes.
Epoch 1/150


C:\Users\sumit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
